In [1]:
import torch
print(torch.cuda.is_available()) 
from Supporting_functions import *

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'        
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

True


# Training SelfNet


In [2]:
# Define paths and paramters

# Models

path =r'\\WS3\WS3_Remote_Disk\Current_Segovia_lab\Deconvolution_Dilan\data\Deconvolution test\SelfNet0\Simulated_Jupyter\Sinusoids/'

batch_size = 10
epochs = 100

min_v = 0
max_v = 65535
pairedDataSets = False

learning_rate_G = 1e-4
learning_rate_D = 1e-4
beta1 = 0.5
beta2 = 0.999
log_interval = 500
imshow_interval = 500

input_nc = 1
output_nc = 1
        
# CUDA device
device=torch.device('cuda:0')

# Create varibale args
class TrainingArgs():
    pass

args = TrainingArgs()
args.path = path
args.batch_size = batch_size
args.epochs = epochs
args.min_v = min_v
args.max_v = max_v
args.learning_rate_G = learning_rate_G
args.learning_rate_D = learning_rate_D
args.beta1 = beta1
args.beta2 = beta2
args.log_interval = log_interval
args.imshow_interval = imshow_interval

In [3]:
# Create folder for output and cheking points

traning_data_path  = os.path.join(path,'train_data/train_data.npz')
checkpoint_path = os.path.join(path,'checkpoint/')

if not os.path.exists(checkpoint_path):
    os.mkdir(checkpoint_path)

model_saved_path = os.path.join(checkpoint_path,'saved_models/')
intermediate_results_path = os.path.join(checkpoint_path,'intermediate_results/')

if not os.path.exists(model_saved_path):
    os.mkdir(model_saved_path)

if not os.path.exists(intermediate_results_path):
    os.mkdir(intermediate_results_path)
    

In [4]:
# Prepare networks
netD_A = Self_net_architecture.define_D(input_nc=input_nc, ndf=64, netD='n_layers', n_layers_D=2, device=device,norm='instance')
netD_B = Self_net_architecture.define_D(input_nc=input_nc, ndf=64, netD='n_layers', n_layers_D=2, device=device,norm='instance')

netG_A = Self_net_architecture.define_G(input_nc=input_nc, output_nc=output_nc, ngf=64, netG='deblur_net', device=device,use_dropout=False,norm='instance')
netG_B = Self_net_architecture.define_G(input_nc=input_nc, output_nc=output_nc, ngf=64, netG='deblur_net', device=device,use_dropout=False,norm='instance')

deblur_net=Self_net_architecture.define_G(input_nc=input_nc, output_nc=output_nc, ngf=64, netG='deblur_net', device=device,use_dropout=False,norm='instance')

# Define loss functions
criterionGAN=Self_net_architecture.GANLoss(gan_mode='lsgan').to(device)
SSIM_loss = SSIM(data_range=1, size_average=True, win_size=11, win_sigma=1.5, channel=1)
criterionL1=L1_Charbonnier_loss()

# Optimizers
optimG = optim.Adam(itertools.chain(netG_A.parameters(), netG_B.parameters()),lr=learning_rate_G, betas=(beta1, beta2))
optimD=optim.Adam(itertools.chain(netD_A.parameters(),netD_B.parameters()),lr=learning_rate_D, betas=(beta1, beta2))
optim_deblur=optim.Adam(deblur_net.parameters(),lr=learning_rate_G, betas=(beta1, beta2))

# Training
train_set=pytorch_dataset.create_train_data(traning_data_path,batch_size,min_v,max_v, pairedDataSets)
train(train_set,netD_A,netD_B,netG_A,netG_B,deblur_net,args,device,criterionGAN,criterionL1,SSIM_loss,optimD,optimG,optim_deblur)
print('done training!')


initialize network with normal
initialize network with normal
initialize network with normal
initialize network with normal
initialize network with normal
done data preprocessing!
epoch:1,learning_rate_D:0.0001
epoch:1,learning_rate_G:0.0001
epochs: 1 5000/10000  degradation modeling stage: loss_D:0.246604 , loss_G: 0.898869 loss_G_cycle:0.198845 loss_feedback:0.000000
epochs: 1 5000/10000  deblurring stage :loss_deblur:0.099489
epochs: 1 10000/10000  degradation modeling stage: loss_D:0.187952 , loss_G: 0.937499 loss_G_cycle:0.166880 loss_feedback:0.000000
epochs: 1 10000/10000  deblurring stage :loss_deblur:0.096203
epoch:2,learning_rate_D:0.0001
epoch:2,learning_rate_G:0.0001
epochs: 2 5000/10000  degradation modeling stage: loss_D:0.207815 , loss_G: 0.813061 loss_G_cycle:0.144449 loss_feedback:0.009394
epochs: 2 5000/10000  deblurring stage :loss_deblur:0.095805
epochs: 2 10000/10000  degradation modeling stage: loss_D:0.216972 , loss_G: 0.770950 loss_G_cycle:0.127408 loss_feedback